In [9]:
using Distributions
using Gadfly
using Interact
using DataFrames

In [10]:
function DistNormal(obs, mean, sd, min, max)
    return rand(Normal(mean, sd),obs)
end

DistNormal (generic function with 1 method)

In [11]:
function DistUniform(obs, mean, sd, min, max)
    return rand(Uniform(min, max),obs)
end

DistUniform (generic function with 1 method)

In [12]:
set_default_plot_size(25cm, 25cm)

In [23]:
@manipulate for samples in 10:50:1000, observations in 10:100:1000, β0 in 0:10, β1 in 0:0.1:1,
    u_min in -10:0, u_max in 0:10, u_mean in -5:5, u_sd in 1:5, u_dist=[:normal => DistNormal, :uniform => DistUniform]
    
    # Create lists for values for intercept and slope OLS estimators
    β0_hat = []
    β1_hat = []
    
    # Bringing x-values outside loop in order to impose less stringent assumption
    x_values = rand(Normal(50,10),observations)
    X = [ones(x_values) x_values]
    
    # Collect values of β among different samples
    for sample in 1:1:samples 
    
        ## Initial
        u = u_dist.(observations, u_mean, u_sd, u_min, u_max)
        #u = DistNormal(observations, u_mean, u_sd, u_min, u_max)
        y_values = β0 + β1 * x_values + u
        
        ## Classical Linear Regression
        β = inv(X'*X)*X'*y_values
        
        ## Save values
        push!(β0_hat, β[1])
        push!(β1_hat, β[2])
    
    end    
    
    # Plot
    
    ## Example values
    u = u_dist.(observations, u_mean, u_sd, u_min, u_max)
    #u = DistNormal(observations, u_mean, u_sd, u_min, u_max)
    y_values = β0 + β1 * x_values + u
    β = inv(X'*X)*X'*y_values
    
    # CLR graph
    graph_CLR = plot(
        layer(x=x_values, y=y_values, Geom.point, order=1),
        layer(x=[0; x_values; 100], y=[X[:1]; X; [1 100]]*β, Geom.line, order=2, Theme(default_color=colorant"black", line_width=1pt)),
        Guide.XLabel("Values of x"),
        Guide.YLabel("Values of y"),
        Guide.Title("Classical Linear Regression"),
        Coord.Cartesian(xmin=0 , xmax=100, ymin=0, ymax=100)
    )
    
    # Distribtuion of β_0
    graph_β_0 = plot(
        layer(x=β0_hat, Geom.histogram(bincount=50), order=1),
        layer(xintercept=[β0], Geom.vline(color=["black"], style=[[1mm]], size=[.5mm]), order=2),
        layer(xintercept=[mean(β0_hat)], Geom.vline(color=["blue"], style=[[1mm]], size=[.5mm]), order=3),
        Guide.XLabel("Estimates of β_0"),
        Guide.YLabel("Number of Observations"),
        Guide.Title("Distribution of β_0"),
        Coord.Cartesian(xmin=0 , xmax=10, ymin=0, ymax = 50)
    )
    
    # Distribtuion of β_1
    graph_β_1 = plot(
        layer(x=β1_hat, Geom.histogram(bincount=50), order=1),
        layer(xintercept=[β1], Geom.vline(color=["black"], style=[[1mm]], size=[.5mm]), order=2),
        layer(xintercept=[mean(β1_hat)], Geom.vline(color=["blue"], style=[[1mm]], size=[.5mm]), order=3),
        Guide.XLabel("Estimates of β_1"),
        Guide.YLabel("Number of Observations"),
        Guide.Title("Distribution of β_1"),
        Coord.Cartesian(xmin=0.35 , xmax=0.65, ymin=0, ymax = 50)
    )
    
    vstack(graph_CLR, hstack(graph_β_0, graph_β_1))
    
end

LoadError: [91mMethodError: no method matching start(::Interact.Options{:SelectionSlider,Int64})[0m
Closest candidates are:
  start([91m::SimpleVector[39m) at essentials.jl:258
  start([91m::Base.MethodList[39m) at reflection.jl:560
  start([91m::ExponentialBackOff[39m) at error.jl:107
  ...[39m

<br>